# Week 3
### Context

#### Experiment Tools
- WanDB

In [ ]:
!nvidia-smi

Sat Aug  7 07:34:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wandb

     |████████████████████████████████| 1.8 MB 8.4 MB/s 
     |████████████████████████████████| 170 kB 37.0 MB/s 
     |████████████████████████████████| 133 kB 40.5 MB/s 
     |████████████████████████████████| 138 kB 40.0 MB/s 
     |████████████████████████████████| 97 kB 7.7 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 62 kB 995 kB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=94d5041164b8efe2fbfe6b85ad824f966993a71039f52bb5a82748560843d212
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=2565da2b64b7d11e092fc5fbe0073b405839dfc8128ab3586a32bb23254395d5
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
  Attempting uninstall: urlli

In [ ]:
sweep_config = {
  "name" : "t-academy_sweep",
  "method" : "bayes",
  "parameters" : {
    "max_depth" : {
      "distribution": "int_uniform",
      "min":2,
      "max":15
    },
    "subsample" :{
      "distribution": "uniform",
      "min": 0.5,
      "max": 1.0
    },
    "colsample_bytree":{
      "distribution": "uniform",
      "min": 0.5,
      "max": 1.0
    }
  },
  "metric":{
      "name": "cv_loss",
      "goal": "minimize"
  }
}


In [ ]:
import os
from os.path import join

import multiprocessing
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import wandb

sweep_id = wandb.sweep(sweep_config, 
                       project="t-academy wandb demo")

n_cpus = multiprocessing.cpu_count()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: a20hskps
Sweep URL: https://wandb.ai/ilevk/t-academy%20wandb%20demo/sweeps/a20hskps


In [ ]:
BASE_DIR = '/content/drive/MyDrive/[2021_07]_T아카데미'

train_path = join(BASE_DIR, 'data', 'MDC14', 'train.csv')
test_path  = join(BASE_DIR, 'data', 'MDC14', 'test.csv')

data = pd.read_csv(train_path)
test = pd.read_csv(test_path)

label = data['credit']

In [ ]:
# 불필요한 컬럼 제거
data.drop(columns=['index', 'credit'], inplace=True)
test.drop(columns=['index'],         inplace=True)

In [ ]:
cat_columns = [c for c, t in zip(data.dtypes.index, data.dtypes) if t == 'O'] 
num_columns = [c for c    in data.columns if c not in cat_columns]

print('Categorical Columns: \n{}\n'.format(cat_columns))
print('Numeric Columns: \n{}'.format(num_columns))

Categorical Columns: 
['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type']

Numeric Columns: 
['child_num', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'FLAG_MOBIL', 'work_phone', 'phone', 'email', 'family_size', 'begin_month']


#### 라벨 데이터 인코딩

In [ ]:
label = label.astype(int)

#### 전처리 프로세스 함수로 작성

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def preprocess(x_train, x_valid, x_test):
    tmp_x_train = x_train.copy()
    tmp_x_valid = x_valid.copy()
    tmp_x_test  = x_test.copy()
    
    tmp_x_train.reset_index(drop=True, inplace=True)
    tmp_x_valid.reset_index(drop=True, inplace=True)
    
    # 결측치 처리
    imputer = SimpleImputer(strategy='most_frequent')
    tmp_x_train[cat_columns] = imputer.fit_transform(tmp_x_train[cat_columns])
    tmp_x_valid[cat_columns] = imputer.transform(tmp_x_valid[cat_columns])
    tmp_x_test[cat_columns]  = imputer.transform(tmp_x_test[cat_columns])
    
    # 스케일링
    scaler = StandardScaler()
    tmp_x_train[num_columns] = scaler.fit_transform(tmp_x_train[num_columns])
    tmp_x_valid[num_columns] = scaler.transform(tmp_x_valid[num_columns])
    tmp_x_test[num_columns]  = scaler.transform(tmp_x_test[num_columns])

    # 인코딩
    ohe = OneHotEncoder(sparse=False)
    ohe.fit(tmp_x_train[cat_columns])
    
    tmp_x_train_cat = pd.DataFrame(ohe.transform(tmp_x_train[cat_columns]))
    tmp_x_valid_cat = pd.DataFrame(ohe.transform(tmp_x_valid[cat_columns]))
    tmp_x_test_cat  = pd.DataFrame(ohe.transform(tmp_x_test[cat_columns]))
    
    tmp_x_train.drop(columns=cat_columns, inplace=True)
    tmp_x_valid.drop(columns=cat_columns, inplace=True)
    tmp_x_test.drop(columns=cat_columns, inplace=True)
    
    tmp_x_train = pd.concat([tmp_x_train, tmp_x_train_cat], axis=1)
    tmp_x_valid = pd.concat([tmp_x_valid, tmp_x_valid_cat], axis=1)
    tmp_x_test  = pd.concat([tmp_x_test, tmp_x_test_cat], axis=1)
    
    return tmp_x_train, tmp_x_valid, tmp_x_test

### Out-of-fold(OOF) Ensemble

In [13]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from xgboost import XGBClassifier

def train():
    with wandb.init() as run:
        params = wandb.config
        
        val_scores = list()
        oof_pred = np.zeros((test.shape[0], 3))
        n_splits = 5

        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

        for i, (trn_idx, val_idx) in enumerate(skf.split(data, label)):
            x_train, y_train = data.iloc[trn_idx, :], label.iloc[trn_idx,]
            x_valid, y_valid = data.iloc[val_idx, :], label.iloc[val_idx,]

            # 전처리
            x_train, x_valid, x_test = preprocess(x_train, x_valid, test)

            # 모델 정의
            model = XGBClassifier(n_estimators=1000,
                                  max_depth=params['max_depth'],
                                  subsample=params['subsample'],
                                  colsample_bytree=params['colsample_bytree'],
                                  tree_method='gpu_hist',
                                  n_jobs=n_cpus-1)

            # 모델 학습
            model.fit(x_train, y_train, 
                      eval_metric='mlogloss', 
                      eval_set=[[x_train, y_train], [x_valid, y_valid]],
                      early_stopping_rounds=100,
                      verbose=100)

            # 훈련, 검증 데이터 log_loss 확인
            trn_logloss = log_loss(y_train, model.predict_proba(x_train))
            val_logloss = log_loss(y_valid, model.predict_proba(x_valid))
            print('{} Fold, train logloss : {:.4f}4, validation logloss : {:.4f}'.format(i, trn_logloss, val_logloss))

            val_scores.append(val_logloss)
            
        metrics = {"cv_loss": np.mean(val_scores)}
        wandb.log(metrics)
count = 5
wandb.agent(sweep_id, function=train, count=count)

wandb: Agent Starting Run: 0fj57bb3 with config:
wandb: 	colsample_bytree: 0.8387914008925683
wandb: 	max_depth: 15
wandb: 	subsample: 0.6186116973904416
wandb: Currently logged in as: ilevk (use `wandb login --relogin` to force relogin)


[0]	validation_0-mlogloss:1.04148	validation_1-mlogloss:1.05192
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 100 rounds.
[100]	validation_0-mlogloss:0.249419	validation_1-mlogloss:0.732797
Stopping. Best iteration:
[52]	validation_0-mlogloss:0.38921	validation_1-mlogloss:0.714112

0 Fold, train logloss : 0.38924, validation logloss : 0.7141
[0]	validation_0-mlogloss:1.04226	validation_1-mlogloss:1.05285
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 100 rounds.
[100]	validation_0-mlogloss:0.247036	validation_1-mlogloss:0.757528
Stopping. Best iteration:
[49]	validation_0-mlogloss:0.404841	validation_1-mlogloss:0.727561

1 Fold, train logloss : 0.40484, validation logloss : 0.7276
[0]	validation_0-mlogloss:1.04094	validation_1-mlogloss:1.05273
Multiple eval metrics have 

cv_loss,0.72251
_runtime,119
_timestamp,1628322697
_step,0


cv_loss,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: zrqmtxgu with config:
wandb: 	colsample_bytree: 0.5630725873190232
wandb: 	max_depth: 3
wandb: 	subsample: 0.9415557607603176


...

cv_loss,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: jaku202w with config:
wandb: 	colsample_bytree: 0.8113843816799957
wandb: 	max_depth: 15
wandb: 	subsample: 0.7076145805064211


[0]	validation_0-mlogloss:1.03885	validation_1-mlogloss:1.05076
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 100 rounds.
[100]	validation_0-mlogloss:0.245812	validation_1-mlogloss:0.731015
Stopping. Best iteration:
[50]	validation_0-mlogloss:0.388743	validation_1-mlogloss:0.711666

0 Fold, train logloss : 0.38874, validation logloss : 0.7117
[0]	validation_0-mlogloss:1.04165	validation_1-mlogloss:1.05276
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 100 rounds.
[100]	validation_0-mlogloss:0.23846	validation_1-mlogloss:0.753631
Stopping. Best iteration:
[53]	validation_0-mlogloss:0.375268	validation_1-mlogloss:0.724076

1 Fold, train logloss : 0.37534, validation logloss : 0.7241
[0]	validation_0-mlogloss:1.04057	validation_1-mlogloss:1.05264
Multiple eval metrics have 

cv_loss,0.72179
_runtime,128
_timestamp,1628323108
_step,0


cv_loss,▁
_runtime,▁
_timestamp,▁
_step,▁
